<span style="font-size:24px;">**Predicting Usage of PlaiCraft**</span>

<span style="font-size:18px;">**Introduction**</span>

A current research study is collecting data about how people play video games on a Minecraft server. The research group conducting this study needs to maximize their recruitment efforts so that they have enough data for their specific research; thus, they have proposed a couple of questions that we, as data science students, will explore and try to answer using our knowledge and the application of data science itself. The question we chose to focus on was, “Which "kinds" of players are most likely to contribute a large amount of data?” In particular, we are going to use data science to predict whether the age and experience level of participants in the study will influence how much time they spend on the server (i.e how much data they contribute). (explain more?)

<span style="font-size:18px;">**Our Question**</span>

Can the age and experience level of participants influence how much time is spent on the server? 

<span style="font-size:18px;">**The Data**</span>



Within this video game research data, there are **two main datasets**:
 
1. **Players.csv** dataset describes general summative information about each participant. The dataset includes 9 columns (variables) and 196 rows (observations). The table below describes each variable within the data set:


| Variable | Type of Variable | Description |
|----------|----------|----------|
| hashedEmail   | character variable (chr) | Participant’s email in unique character code |
| experience	| character variable (chr) | Participant’s previous Minecraft experience |
| subscribe	| logical variable (lgl) | If participant is subscribed to PlaiCraft’s email updates |
| played_hours	| double variable (dbl) | Number of hours played by each participant |
| name	| character variable (chr) | Name of participant |
| gender	| character variable (chr) | Gender of participant |
| age   | double variable (dbl) | Age of participant |
| individualID	| logical variable (lgl) | ID of participant |
| organizationName	| logical variable (lgl) | Organization of participant |


2. **Sessions.csv** focuses on each participant’s play time sessions.  This dataset includes 5 columns (variables) and currently 1535 rows (observations). Below describes each variable within the data set:
 
| Variable | Type of Variable | Description |
|----------|----------|----------|
| hashedEmail   | character variable (chr) | Participant’s email in unique character code |
| start_time	| character variable (chr) | Date and start time of a play session for each participant |
| end_time	| character variable (chr) | Date and end time of a play session for each participant |
| original_start_time	| double variable (dbl) | Scheduled start time in seconds since January 1st, 1970 |
| original_end_time	| double variable (dbl) | Scheduled end time in seconds since January 1st, 1970 |


In this report, we will be using **dataset #1**, although both can be important; our question requires further analysis of dataset #1, **players.csv**. Before we begin using the dataset, let's evaluate some potential issues with the selected data set. 

Potential Issues within this dataset:

    - Experience variable should be stored as a factor variable since it refers to a specific set of catergories
    - IndividualID & OrganizationName are missing values, thus unclear if these variables are signficant
    - Age of participants may be unaccurate as participant's could give false information
    - Privacy concerns regarding participant's name and email

<span style="font-size:18px;">**Beginning Our Analysis**</span>

Before we begin using our dataset to predict and answer our question at hand. We first need to do the necessary steps to load in our data and tidy it up so our functions recognize and can use it.

Let's begin by reading/loading in our data. Before that, let's load the necessary packages that have the functions we can use to actually read in our data.

In [3]:
### Run this cell before continuing. These library packages in R give us functions we can us
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
library(tidyr)
library(dplyr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

Now we can read in our dataset.

In [6]:
#Since our dataset comes from a url link, we must read it in this way.
url_1 <- "https://drive.google.com/uc?export=download&id=1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz"
players_data <- read_csv(url_1)
players_data

Rows: 196 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, age
lgl (3): subscribe, individualId, organizationName

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,NA,NA
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17,NA,NA
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17,NA,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db299bd4fedb06a46ad5bb,0.0,Dylan,Prefer not to say,17,NA,NA
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17,NA,NA
Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,91,NA,NA


Referring back to our question, we want the specific columns **age**, **experience**, and **played_hours**. So let's select these specific columns and name our new data set **players_data_selected**.

In [8]:
players_data_selected <- players_data |>
    select(age, experience, played_hours)
players_data_selected

age,experience,played_hours
<dbl>,<chr>,<dbl>
9,Pro,30.3
17,Veteran,3.8
17,Veteran,0.0
⋮,⋮,⋮
17,Amateur,0.0
17,Amateur,2.3
91,Pro,0.2
